## Testing out the PuLP linear programming package.  We'll work through some examples. Ultimately, I want to compare this to ORTools.

In [1]:
# Import modules

from __future__ import division

import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from tqdm import tqdm
from pulp import *

## The "miracle worker problem"
* A 'miracle' worker is trying to optimize the production of two medicines.  
* To create one unit of medicine 1, you need 3 units of herb A and 2 units of herb B.
* To create one unit of medicine 2, you need 4 units of herb A and 1 unit of herb B.
* Medicine 1 can heal a person by 25 units of health.
* Medicine 2 can heal a person by 20 units of health.
* You have 25 units of herb A and 10 units of herb B.
* What quantity of medicines 1 and 2 should you make to maximize how much you can heal the next patient?

In [2]:
# Create the 'prob' variable.  This will contain the problem data and definition.
prob = LpProblem("The mircale worker", LpMaximize)

In [4]:
# Define the variables.  The units of medicine 1 produced will be variable x, and the units of medicine 2 produced with be variable y.
# The variables here are going to be integers (so the output, which is the last argument below, is an interger type).
# The second argument is the lower bound (can't have negative amounts of medicine).
# The third argument is the upper bound (which could be anything here really).
x = LpVariable("medicine_1_units", 0, None, LpInteger)
y = LpVariable('medicine_2_units', 0, None, LpInteger)

In [6]:
# Define the objective function.  We want to maximize the amount we can heal, so per the problem definition it must be 25*x + 20*y = health restored.
# We define this by adding it to prob
prob += 25*x + 20*y, "health restored; to be maximized"

In [7]:
# Add the supply constraints for the herbs needed to make the medicines.
# If we produce x of medicine 1 and y of medicine 2, we need 3 units of herb A to make medicine 1 and 4 units of herb B to make medicine 2. So we use 3*x + 4*y units of herb A.
# Similarly, we use 2*x + y units of herb B.
# We have 25 units of herb A and 10 units of herb B in supply, so the constraints on supply are:
prob += 3*x + 4*y <= 25, "Herb A constraint"
prob += 2*x + y <= 10, "Herb B constraint"

In [8]:
# We can write out the problem with definitions to a .lp file. This was we can reload and solve the problem without having to redefine everything.
prob.writeLP("miracle_worker.lp")

In [9]:
# Run the solver (using the PuLP default)
prob.solve()

1

In [10]:
# Get the solver status
print("Status: {0}".format(LpStatus[prob.status]))

Status: Optimal


In [12]:
# So we've got the optimal solution.
# We can get back each variable with its optimal value
for v in prob.variables():
    print("{0} = {1}".format(v.name, v.varValue))

medicine_1_units = 3.0
medicine_2_units = 4.0


In [13]:
# So the best course of action is to produce 3 units of medicine 1 and 4 units of medicine 2.
# The maximum amount of health that can be restored (from producing the above amount of medicines 1 and 2) is:
print('Total health that can be restored = {0}'.format(value(prob.objective)))

Total health that can be restored = 155.0


## A blending problem
* The problem statement: We want to produce cat food as cheaply as possible, while ensuring we meet a suitable standard for nutrition.
* Data: The main ingredients are chicken, beef, mutton, rice, wheat, gel.
    * The cost for each ingredient per gram is:
        * Chicken: $0.010
        
        * Beef: $0.008
        
        * Mutton: $0.010
        
        * Rice: $0.002
        
        * Wheat: $0.005
        
        * Gel: $0.001
    * The contributions per gram for each ingredients towards nutrition are:
        * Chicken:
            * Protein: 0.100
            * Fat: 0.080
            * Fiber: 0.001
            * Salt: 0.002
        * Beef:
            * Protein: 0.200
            * Fat: 0.100
            * Fiber: 0.005
            * Salt: 0.005
        * Mutton:
            * Protein: 0.150
            * Fat: 0.110
            * Fiber: 0.003
            * Salt: 0.007
        * Rice:
            * Protein: 0.000
            * Fat: 0.010
            * Fiber: 0.100
            * Salt: 0.002
        * Wheat bran:
            * Protein: 0.040
            * Fat: 0.010
            * Fiber: 0.150
            * Salt: 0.008
        * Gel:
            * Protein: 0.000
            * Fat: 0.000
            * Fiber: 0.000
            * Salt: 0.000
            
    * The total amount of material per can of cat food is 100 grams.
    
    * The nutritional requirements per can of cat food (i.e. per 100g) are:
        * Protein: at least 8g
        * Fat: at least 6g
        * Fiber: no more than 2g
        * Salt: no more than 0.4g

## Model the problem:
* Identify the variables. These are the percentages of the different ingredients we include in the can.  Since each can is 100g, the variables can be expressed as the amount of each ingredient in grams.  
    * x_1 = percentage of chicken in a can of cat food.
    * x_2 = percentage of beef in a can of cat food.
    * x_3 = percentage of mutton in a can of cat food.
    * x_4 = percentage of rice in a can of cat food.
    * x_5 = percentage of wheat in a can of cat food.
    * x_6 = percentage of gel in a can of cat food.

* Each of these variables is a percentage, and so are restricted to be between 0 and 100.

* The objective is to minimize the cost per can of cat food.  Since the can is 100 grams, the cost is just the x_i variables times the cost per gram. Otherwise, if each can was n grams, we would compute (x_i/100) * n * cost_per_gram_x_i.  The objective is then:
    * min(0.013*x_1 + 0.008*x_2 + 0.010*x_3 + 0.002*x_4 + 0.005*x_5 + 0.001*x_6)
    
* The constraints:
    * The sum of the percentages must add up to 100:
        * x_1 + x_2 + x_3 + x_4 + x_5 + x_6 = 100
    * The nutritional requirements are expressed as:
        * 0.100*x_1 + 0.200*x_2 + 0.150*x_3 + 0.000*x_4 + 0.040*x_5 + 0.0*x_6 >= 8.0
        * 0.080*x_1 + 0.100*x_2 + 0.110*x_3 + 0.010*x_4 + 0.010*x_5 + 0.0*x_6 >= 6.0
        * 0.001*x_1 + 0.005*x_2 + 0.003*x_3 + 0.100*x_4 + 0.150*x_5 + 0.0*x_6 <= 2.0
        * 0.002*x_1 + 0.005*x_2 + 0.007*x_3 + 0.002*x_4 + 0.008*x_5 + 0.0*x_6 <= 0.4

In [15]:
# Build a dataframe with all of the relevant data.
data_dict = {'CHICKEN': {'cost_per_gram': 0.013, 'protein_per_gram': 0.100, 'fat_per_gram': 0.080, 'fiber_per_gram': 0.001, 'salt_per_gram': 0.002}, 
             'BEEF': {'cost_per_gram': 0.008, 'protein_per_gram': 0.200, 'fat_per_gram': 0.100, 'fiber_per_gram': 0.005, 'salt_per_gram': 0.005},
             'MUTTON': {'cost_per_gram': 0.010, 'protein_per_gram': 0.150, 'fat_per_gram': 0.110, 'fiber_per_gram': 0.003, 'salt_per_gram': 0.007},
             'RICE': {'cost_per_gram': 0.002, 'protein_per_gram': 0.000, 'fat_per_gram': 0.010, 'fiber_per_gram': 0.100, 'salt_per_gram': 0.002},
             'WHEAT': {'cost_per_gram': 0.005, 'protein_per_gram': 0.040, 'fat_per_gram': 0.010, 'fiber_per_gram': 0.150, 'salt_per_gram': 0.008},
             'GEL': {'cost_per_gram': 0.001, 'protein_per_gram': 0.000, 'fat_per_gram': 0.000, 'fiber_per_gram': 0.000, 'salt_per_gram': 0.000}}

In [18]:
data_df = pd.DataFrame.from_dict(data_dict, orient='index')

In [19]:
data_df

,protein_per_gram,fiber_per_gram,salt_per_gram,fat_per_gram,cost_per_gram
BEEF,0.20,0.005,0.005,0.10,0.008
CHICKEN,0.10,0.001,0.002,0.08,0.013
GEL,0.00,0.000,0.000,0.00,0.001
MUTTON,0.15,0.003,0.007,0.11,0.010
RICE,0.00,0.100,0.002,0.01,0.002
WHEAT,0.04,0.150,0.008,0.01,0.005


In [20]:
# Define the problem variable. We want to minimize the cost objective here.
prob = LpProblem("Cat food optimization", LpMinimize)

In [23]:
# Create a dictionary to contained the variables by name
ingredient_vars = LpVariable.dicts("Ingr", data_df.index.tolist(), 0)

In [26]:
# The costs can then be added as constraints, since the indgredients dictionary has keys equal to the data_df indices:
prob += lpSum([data_df['cost_per_gram'].to_dict()[i] * ingredient_vars[i] for i in data_df.index.tolist()]), "Total cost of ingredients per can"

In [27]:
# Add the five constraints to the problem definition.
prob += lpSum([ingredient_vars[i] for i in data_df.index.tolist()]) == 100, "Percentages sum"
prob += lpSum([data_df['protein_per_gram'].to_dict()[i] * ingredient_vars[i] for i in data_df.index.tolist()]) >= 8.0, "Protein requirement"
prob += lpSum([data_df['fat_per_gram'].to_dict()[i] * ingredient_vars[i] for i in data_df.index.tolist()]) >= 6.0, "Fat requirement"
prob += lpSum([data_df['fiber_per_gram'].to_dict()[i] * ingredient_vars[i] for i in data_df.index.tolist()]) <= 2.0, "Fiber requirement"
prob += lpSum([data_df['salt_per_gram'].to_dict()[i] * ingredient_vars[i] for i in data_df.index.tolist()]) <= 0.4, "salt requirement"

In [28]:
# Write out the problem.
prob.writeLP('CatFoodModel.lp')

In [29]:
# Solve the problem using default solver.
prob.solve()

1

In [30]:
# Get the solution status
print("Status: {0}".format(LpStatus[prob.status]))

Status: Optimal


In [31]:
# So we have an optimal solution.
# Look at the optimal value of each variable.
for v in prob.variables():
    print("{0} = {1}".format(v.name, v.varValue))

Ingr_BEEF = 60.0
Ingr_CHICKEN = 0.0
Ingr_GEL = 40.0
Ingr_MUTTON = 0.0
Ingr_RICE = 0.0
Ingr_WHEAT = 0.0


In [32]:
# Interesting.  What is the cost per can?
print('Total cost of ingredients per can: {0}'.format(value(prob.objective)))

Total cost of ingredients per can: 0.52


In [ ]:
# So it's 60% beed and 40% "gel"...sounds terrible.  